<a href="https://colab.research.google.com/github/NLP-END3/Session3/blob/main/Session3_Pytorch101_ver3a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext tensorboard

## 1. Problem Statement

Write a neural network that can:

1. take 2 inputs:
    - an image from the MNIST dataset (say 5), and
    - a random number between 0 and 9, (say 7)
2. and gives two outputs:
    - the "number" that was represented by the MNIST image (predict 5), and
    - the "sum" of this number with the random number and the input image to the network (predict 5 + 7 = 12)
3. you can mix fully connected layers and convolution layers
4. you can use one-hot encoding to represent the random number input as well as the "summed" output.  
    a. Random number (7) can be represented as 0 0 0 0 0 0 0 1 0 0  
    b. Sum (13) can be represented as: 1. 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0  
    c. 0b1101 (remember that 4 digits in binary can at max represent 15, so we may need to go for 5 digits. i.e. 10010  

## 2. Importing required libraries & Checking GPU


In [2]:
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR

print(torch.cuda.is_available())  # Checks if GPU is available
print(torch.cuda.get_device_name(0)) # Name of GPU
print(torch.cuda.device_count())

False


RuntimeError: ignored

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/mnist_experiment_1')

## 3. Importing MNIST dataset from pytorch

In [ ]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
        ])

mnist_train = datasets.MNIST('../data',train=True,download=True) # Train dataset
mnist_test = datasets.MNIST('./data',train=False,download=True) # Test dataset

## 3. Plotting few samples of the downloaded data

In [ ]:
figure = plt.figure(figsize=(8, 8))
cols, rows = 10, 10
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(mnist_train), size=(1,)).item()
    img, label = mnist_train[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(label)
    plt.axis("off")
    plt.imshow(img, cmap="gray")
figure.tight_layout()
plt.show()

In [ ]:
#dir(mnist_train)

## 4. Checking the size of the image, label and image information

In [ ]:
print(f'Number of examples in training dataset :{len(mnist_train)}')
print(f'Shape of the training dataset - images : {mnist_train.data.shape}')
print(f'Labels in the training dataset : {mnist_train.targets}')

## 5. Defining Custom Dataset Class

In [ ]:
from torch.utils.data import Dataset
from random import randrange

# Dataset is there to be able to interact with DataLoader

class MyDataset(Dataset):
  def __init__(self, inpDataset, transform):
    self.inpDataset = inpDataset
    self.transform = transform

  def __getitem__(self, index):
    randomNumber = randrange(10)
    sample_image, label = self.inpDataset[index]
    if self.transform:
        sample_image = self.transform(sample_image)

    sample = (sample_image,F.one_hot(torch.tensor(randomNumber),num_classes=10), label,label+randomNumber)
    return sample

  def __len__(self):
    return len(self.inpDataset)

myData_train = MyDataset(mnist_train,transform) 
myData_test = MyDataset(mnist_test,transform) 

In [ ]:
image,randomNumber, label1, label2 = next(iter(myData_train))
image.shape,randomNumber, label1, label2

## 6. Creating DataLoader

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda") if use_cuda else torch.device("cpu")

train_kwargs = {'batch_size': 1000}
test_kwargs = {'batch_size': 1000}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

train_loader = torch.utils.data.DataLoader(myData_train,**train_kwargs)
test_loader = torch.utils.data.DataLoader(myData_test, **test_kwargs)

# train_loader = torch.utils.data.DataLoader(mnist_train,**train_kwargs)
# test_loader = torch.utils.data.DataLoader(mnist_test, **test_kwargs)

In [ ]:
# helper function to show an image
# (used in the `plot_classes_preds` function below)
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
# get some random training images
dataiter = iter(train_loader)
images, randomNumber, labels, sum = dataiter.next()

# create grid of images
img_grid = torchvision.utils.make_grid(images[0:100,])

# show images
matplotlib_imshow(img_grid, one_channel=True)

# write to tensorboard
writer.add_image('mnist_images', img_grid)

In [ ]:
%tensorboard --logdir=runs

## 7. Defining Network

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9226, 128)
        self.fc2 = nn.Linear(128, 10)
        self.fc3 = nn.Linear(128, 20)

    def forward(self, x,y):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = torch.cat((x, y), 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x1 = self.fc2(x)
        x2 = self.fc3(x)
        output1 = F.log_softmax(x1, dim=1)
        output2 = F.log_softmax(x2, dim=1)
        return output1, output2

In [ ]:
model = Net().to(device)

In [ ]:
model

## 8. Training Network

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data,randomNumber,target,target1) in enumerate(train_loader):
        data,randomNumber,target,target1 = data.to(device), randomNumber.to(device), target.to(device), target1.to(device)
        optimizer.zero_grad()
        output, output1 = model(data,randomNumber)
        loss = F.nll_loss(output, target) + F.nll_loss(output1, target1) * 2
        loss.backward()
        optimizer.step()
        log_interval = 10
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            writer.add_scalar('training loss',
                            loss / log_interval,
                            epoch * len(train_loader) + batch_idx)


def test(model, device, test_loader,epoch):
    model.eval()
    test_loss = 0
    correct = 0
    correct1 = 0
    with torch.no_grad():
        for data,randomNumber,target,target1 in test_loader:
            data,randomNumber,target,target1 = data.to(device), randomNumber.to(device), target.to(device), target1.to(device)
            output, output1 = model(data,randomNumber)
            test_loss += F.nll_loss(output, target, reduction='sum').item() + F.nll_loss(output1, target1, reduction='sum').item() * 2  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            pred1 = output1.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            correct1 += pred1.eq(target1.view_as(pred1)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Image Accuracy: {}/{} ({:.0f}%), Sum Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset), correct1, len(test_loader.dataset),
        100. * correct1 / len(test_loader.dataset)))
    
    writer.add_scalar('test loss',
                       test_loss,
                        epoch)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.1,  momentum=0.9)
epochs = 20
# scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader, epoch)
    # scheduler.step()